# 03) 멀티오믹스 상관분석 (mRNA ↔ Methylation)

## 실습 목표
1) 공통 샘플 정렬 및 부분집합 선택(고변이 기준)
2) Spearman 상관: 단일 예시 → 전체(부분집합) 전수 계산
3) 상관계수 분포/Top hit 테이블/산점도
4) 상관 히트맵 (gene × probe)
5) O2PLS-like

## 패키지 설치 안내
이 스크립트는 UV를 사용하여 자동으로 필요한 패키지를 설치합니다.
수동 설치가 필요한 경우 아래 명령어를 사용하세요:

```bash
pip install numpy pandas matplotlib seaborn scikit-learn scipy
```

## 필수 라이브러리 임포트
- 멀티오믹스 데이터 통합 분석을 위한 고급 라이브러리들을 로드합니다.
- 상관분석, 차원축소, 캐노니컬 분석을 위한 도구들을 준비합니다.

In [ ]:
import os, numpy as np, pandas as pd, seaborn as sns, matplotlib.pyplot as plt
import math
from scipy.stats import spearmanr, rankdata
from sklearn.preprocessing import StandardScaler
from sklearn.cross_decomposition import CCA, PLSCanonical
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings('ignore')

# Codespace 작업 디렉토리 설정
os.chdir('/workspaces/pnu/practice_omics')

# 한글 폰트 설정 (시스템에 따라 자동 설정)
import platform
system = platform.system()
if system == 'Darwin':  # macOS
    plt.rc('font', family='AppleGothic')
elif system == 'Windows':
    plt.rc('font', family='Malgun Gothic')
else:  # Linux
    import matplotlib.font_manager as fm
    font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
    if os.path.exists(font_path):
        font_prop = fm.FontProperties(fname=font_path)
        plt.rc('font', family=font_prop.get_name())
    else:
        # DejaVu Sans를 fallback으로 사용
        plt.rc('font', family='DejaVu Sans')

# 마이너스 기호 깨짐 방지
plt.rc('axes', unicode_minus=False)

# 시각화 스타일 설정
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (11, 7)
np.random.seed(42)

print("멀티오믹스 상관분석 시작!")
print("=" * 50)

## 데이터 경로 설정
- 분석에 필요한 mRNA, Clinical, Methylation 파일 경로를 설정합니다.

In [ ]:
MRNA  = "mrna_matched.txt"
CLIN  = "clinical_matched.tsv"
METH  = "methylation_matched.txt"  
SAMPLE_COL = "Sample ID"

print("파일 경로 설정 완료")

## 멀티오믹스 데이터 로드 및 정렬
- 세 가지 오믹스 데이터를 로드하고 공통 샘플로 정렬합니다.
- 교집합 연산을 통해 모든 데이터에 공통으로 존재하는 샘플만 선별합니다.

In [ ]:
print("\n멀티오믹스 데이터 로드 중...")

mrna    = pd.read_csv(MRNA,  sep="\t", index_col=0)
clinical= pd.read_csv(CLIN,  sep="\t")
methyl  = pd.read_csv(METH,  sep="\t", index_col=0)  

# 공통 샘플 정렬 (교집합)
samples = sorted(set(mrna.columns) & set(clinical[SAMPLE_COL]) & set(methyl.columns))

mrna     = mrna.loc[:, samples]
methyl   = methyl.loc[:, samples]
clinical = clinical[clinical[SAMPLE_COL].isin(samples)].reset_index(drop=True)

print("데이터 로드 완료!")
print(f"mRNA: {mrna.shape} | methyl: {methyl.shape} | clinical: {clinical.shape}")
print(f"공통 샘플 수: {len(samples)}개")

## 부분집합 크기 설정 (메모리 및 속도 최적화)
- 대용량 데이터에서도 빠른 상호작용 분석을 위해 상위 변이 기준으로 부분집합을 생성합니다.
- 유전자와 프로브 개수를 조절하여 계산 복잡도를 관리합니다.

In [ ]:
print("\n분석 부분집합 설정 중...")

# 상위 변이 피처 개수 (필요시 조정 가능)
N_GENES  = 100
N_PROBES = 1000

print(f"선별 기준: 상위 {N_GENES}개 유전자 × 상위 {N_PROBES}개 프로브")

# 변이(분산) 기준으로 상위 유전자 / 프로브 선택
top_genes  = mrna.var(axis=1).sort_values(ascending=False).head(N_GENES).index
top_probes = methyl.var(axis=1).sort_values(ascending=False).head(N_PROBES).index

# 서브셋 추출
A = mrna.loc[top_genes]     # (G x n_samples)
B = methyl.loc[top_probes]  # (P x n_samples)

print("부분집합 생성 완료!")
print(f"유전자: {A.shape[0]} × 샘플: {A.shape[1]} | 프로브: {B.shape[0]} × 샘플: {B.shape[1]}")

## Spearman 상관분석 단일 예시
- 실습자가 직접 유전자와 프로브를 선택하여 즉시 상관관계를 확인할 수 있습니다.
- 산점도와 상관계수를 통해 직관적인 관계를 파악합니다.

In [ ]:
print("\nSpearman 상관분석 예시 실행...")

gene_ex  = A.index[10]      # ← 이 값을 바꿔보세요 (예: 0~99)
probe_ex = B.index[20]      # ← 이 값을 바꿔보세요 (예: 0~999)

print(f"선택된 유전자: {gene_ex}")
print(f"선택된 프로브: {probe_ex}")

rho, p = spearmanr(A.loc[gene_ex], B.loc[probe_ex])
print(f"Spearman 상관계수: ρ = {rho:.3f}, p-value = {p:.2e}")

# 산점도 시각화
plt.figure(figsize=(8, 6))
plt.scatter(A.loc[gene_ex], B.loc[probe_ex], alpha=0.7, s=40, color='steelblue', edgecolors='white', linewidth=0.5)
plt.title(f"Gene-Probe Correlation\n{gene_ex} vs {probe_ex} (ρ = {rho:.3f})")
plt.xlabel("mRNA Expression Level")
plt.ylabel("Methylation β Value")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("단일 예시 분석 완료!")

### Vibe Coding 미션 D1: 상관분석 대화형 탐색
**미션**: 여러 유전자-프로브 쌍의 상관관계를 한번에 탐색할 수 있는 도구를 만들어보세요!

**도전 과제**:
1. 랜덤하게 선택된 9개 쌍의 상관관계를 3×3 서브플롯으로 표시하기
2. 각 서브플롯에 상관계수와 유의성 정보 표시하기
3. 강한 양의 상관, 강한 음의 상관, 무상관 쌍들을 색깔로 구분하기


In [ ]:
# 미션 D1 코드를 여기에 작성하세요!


## 대규모 Spearman 상관계수 행렬 계산
- 모든 유전자-프로브 쌍에 대한 상관계수를 효율적으로 계산합니다.
- 순위 변환과 표준화를 통한 최적화된 행렬 연산을 사용합니다.

In [ ]:
print("\n전체 상관계수 행렬 계산 중...")

def row_rank_standardize(M: np.ndarray) -> np.ndarray:
    """
    효율적인 Spearman 상관계수 계산을 위한 순위 변환 및 표준화
    
    Parameters:
    - M: (n_features x n_samples) 행렬
    
    Returns:
    - Z: 표준화된 순위 행렬 (평균=0, 표준편차=1)
    """
    print("순위 변환 중...")
    # 각 행(피처)별로 순위 계산 (동점은 평균 순위 사용)
    ranks = np.apply_along_axis(lambda x: rankdata(x, method="average"), 1, M)
    
    print("표준화 적용 중...")
    # 표준화 (평균=0, 표준편차=1)
    ranks = ranks - ranks.mean(axis=1, keepdims=True)
    denom = ranks.std(axis=1, ddof=1, keepdims=True)
    denom[denom == 0] = 1.0  # 0으로 나누기 방지
    Z = ranks / denom
    return Z

# numpy 배열로 변환 및 결측치 처리
A_np = A.to_numpy(dtype=float)   # (G x n)
B_np = B.to_numpy(dtype=float)   # (P x n)

print("결측치 처리 중...")
# 결측치를 행별 평균으로 대체
A_np = np.where(np.isnan(A_np), np.nanmean(A_np, axis=1, keepdims=True), A_np)
B_np = np.where(np.isnan(B_np), np.nanmean(B_np, axis=1, keepdims=True), B_np)

# 순위 변환 및 표준화
ZA = row_rank_standardize(A_np)  # (G x n)
ZB = row_rank_standardize(B_np)  # (P x n)

print("상관계수 행렬 계산 중...")
# Spearman 상관행렬: ZA @ ZB^T / (n-1)
n = ZA.shape[1]
spearman_mat = (ZA @ ZB.T) / (n - 1)          # (G x P)

print("상관계수 행렬 계산 완료!")
print(f"행렬 크기: {spearman_mat.shape}")
print(f"상관계수 범위: [{spearman_mat.min():.3f}, {spearman_mat.max():.3f}]")

### Vibe Coding 미션 D2: 상관계수 계산 최적화 비교
**미션**: 서로 다른 상관계수 계산 방법들의 성능과 정확도를 비교해보세요!

**도전 과제**:
1. scipy.stats.spearmanr()과 직접 구현한 방법의 결과 비교하기
2. 작은 부분집합에서 계산 시간 측정 및 비교하기
3. 메모리 사용량이 계산 시간에 미치는 영향 분석하기


In [ ]:
# 미션 D2 코드를 여기에 작성하세요!


## 상관계수 분포 분석 및 Top Hits 식별
- 전체 상관계수의 분포를 히스토그램으로 시각화합니다.
- 강한 양의 상관과 강한 음의 상관을 보이는 상위 쌍들을 식별합니다.

In [ ]:
print("\n상관계수 분포 분석 중...")

# 전체 상관계수 분포 시각화
rho_flat = spearman_mat.ravel()
plt.figure(figsize=(10, 6))
sns.histplot(rho_flat, bins=50, kde=True, alpha=0.7, color='skyblue')
plt.axvline(-0.5, ls="--", c="tab:red", linewidth=2, label="Strong Negative (ρ < -0.5)")
plt.axvline(0.5, ls="--", c="tab:green", linewidth=2, label="Strong Positive (ρ > 0.5)")
plt.axvline(0, ls="-", c="gray", alpha=0.5, label="No Correlation")
plt.title("Distribution of Spearman Correlation Coefficients")
plt.xlabel("Correlation Coefficient (ρ)")
plt.ylabel("Frequency")
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("분포 분석 완료")
print(f"강한 양의 상관 (ρ > 0.5): {(rho_flat > 0.5).sum():,}쌍")
print(f"강한 음의 상관 (ρ < -0.5): {(rho_flat < -0.5).sum():,}쌍")

In [ ]:
# Top-N 상관쌍 추출
print("\nTop 상관쌍 식별 중...")
TOPN = 30
G, P = spearman_mat.shape

# 효율적인 top-k 선택을 위한 argpartition 사용
flat_idx_pos = np.argpartition(-spearman_mat, TOPN, axis=None)[:TOPN]  # 가장 큰 값들
flat_idx_neg = np.argpartition(spearman_mat, TOPN, axis=None)[:TOPN]   # 가장 작은 값들

# 인덱스를 2D 좌표로 변환
gi_pos, pi_pos = np.unravel_index(flat_idx_pos, (G, P))
gi_neg, pi_neg = np.unravel_index(flat_idx_neg, (G, P))

# Positive Top-N 테이블
top_pos = (
    pd.DataFrame({
        "Gene": A.index[gi_pos],
        "Probe": B.index[pi_pos],
        "Correlation": spearman_mat[gi_pos, pi_pos]
    })
    .sort_values("Correlation", ascending=False)
    .reset_index(drop=True)
)

# Negative Top-N 테이블  
top_neg = (
    pd.DataFrame({
        "Gene": A.index[gi_neg],
        "Probe": B.index[pi_neg],
        "Correlation": spearman_mat[gi_neg, pi_neg]
    })
    .sort_values("Correlation", ascending=True)
    .reset_index(drop=True)
)

print("Top 상관쌍 식별 완료!")
print("\nTop 10 양의 상관:")
for i, row in top_pos.head(10).iterrows():
    print(f"   {i+1:2d}. {row['Gene']} ↔ {row['Probe']}: ρ = {row['Correlation']:.3f}")

print("\nTop 10 음의 상관:")
for i, row in top_neg.head(10).iterrows():
    print(f"   {i+1:2d}. {row['Gene']} ↔ {row['Probe']}: ρ = {row['Correlation']:.3f}")

### Vibe Coding 미션 D3: Top Hits 심화 분석
**미션**: 발견된 상위 상관쌍들의 생물학적 의미를 탐색해보세요!

**도전 과제**:
1. Top 양의 상관과 음의 상관 쌍들을 2×5 서브플롯으로 시각화하기
2. 각 유전자가 몇 개의 프로브와 강한 상관을 보이는지 카운팅하기
3. 네트워크 그래프로 유전자-프로브 상관 관계 시각화하기

In [ ]:
# 미션 D3 코드를 여기에 작성하세요!


## Top 상관쌍 산점도 시각화
- 발견된 상위 상관쌍들을 산점도로 시각화하여 관계를 직관적으로 확인합니다.
- 양의 상관과 음의 상관을 함께 표시하여 패턴을 비교합니다.

In [ ]:
print("\nTop 상관쌍 시각화 중...")

# 공통 샘플 확인
common_samples = sorted(set(mrna.columns) & set(methyl.columns))

# 시각화할 쌍 선별 (양/음 상위 3개씩, 중복 제거)
pairs_df = pd.concat([
    top_pos.head(3)[["Gene", "Probe"]],
    top_neg.head(3)[["Gene", "Probe"]]
], axis=0).drop_duplicates()

pairs = list(pairs_df.itertuples(index=False, name=None))
print(f"시각화할 쌍 수: {len(pairs)}개")

# 2열 그리드로 산점도 생성
cols = 2
rows = int(np.ceil(len(pairs) / cols))
plt.figure(figsize=(6*cols, 4*rows))

for i, (gene, probe) in enumerate(pairs, 1):
    # 중복 인덱스 처리를 위한 안전한 데이터 추출
    x = mrna.loc[[gene], common_samples].mean(axis=0)
    y = methyl.loc[[probe], common_samples].mean(axis=0)
    
    # 결측치 제거
    valid = x.notna() & y.notna()
    xv, yv = x[valid].to_numpy(), y[valid].to_numpy()
    
    # 상관계수 재계산
    r, p = spearmanr(xv, yv) if len(xv) >= 3 else (0.0, 1.0)
    
    # 서브플롯 생성
    plt.subplot(rows, cols, i)
    
    # 상관계수에 따른 색상 선택
    color = 'red' if r > 0 else 'blue'
    plt.scatter(xv, yv, alpha=0.6, s=30, color=color, edgecolors='white', linewidth=0.5)
    
    plt.title(f"{gene} ↔ {probe}\nρ = {r:.3f}", fontsize=10)
    plt.xlabel("mRNA Expression")
    plt.ylabel("Methylation β")
    plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Top 상관쌍 시각화 완료!")

### Vibe Coding 미션 D4: 고급 상관관계 시각화
**미션**: 상관관계를 더욱 정교하고 인사이트가 풍부한 방식으로 시각화해보세요!

**도전 과제**:
1. 산점도에 회귀직선과 신뢰구간 추가하기
2. 암 아형별로 점들을 다른 색상과 모양으로 구분하기
3. 상관계수의 유의성에 따라 제목 색상 변경하기


In [ ]:
# 미션 D4 코드를 여기에 작성하세요!


## 상관계수 히트맵 시각화
- 계산 비용을 고려하여 부분집합 히트맵을 생성합니다.
- 유전자와 프로브 간의 전체적인 상관 패턴을 한눈에 파악할 수 있습니다.

In [ ]:
print("\n상관계수 히트맵 생성 중...")

# 히트맵용 소형 부분집합 (메모리/시간 절약)
G_hm = min(30, A.shape[0])
P_hm = min(200, B.shape[0])

print(f"히트맵 크기: {G_hm} genes × {P_hm} probes")

# 상위 변이 기준으로 추가 선별
genes_hm  = A.var(axis=1).sort_values(ascending=False).head(G_hm).index
probes_hm = B.var(axis=1).sort_values(ascending=False).head(P_hm).index

# 라벨이 있는 DataFrame으로 변환
spearman_df = pd.DataFrame(spearman_mat, index=A.index, columns=B.index)

# 라벨 기반 슬라이싱
hm_df = spearman_df.loc[genes_hm, probes_hm]

# 중복 라벨 처리 (평균 집계)
hm_df = hm_df.groupby(level=0).mean() if hm_df.index.duplicated().any() else hm_df
hm_df = hm_df.T.groupby(level=0).mean().T if hm_df.columns.duplicated().any() else hm_df

# 히트맵 생성
plt.figure(figsize=(14, 8))
sns.heatmap(hm_df, 
            cmap="RdBu_r", 
            center=0, 
            vmin=-1, 
            vmax=1,
            cbar_kws={'label': 'Spearman ρ', 'shrink': 0.8})
plt.title("Gene-Probe Correlation Heatmap")
plt.xlabel("Methylation Probes")
plt.ylabel("mRNA Genes")
plt.xticks(rotation=45, ha='right', fontsize=8)
plt.yticks(rotation=0, fontsize=8)
plt.tight_layout()
plt.show()

print("상관계수 히트맵 생성 완료!")

### Vibe Coding 미션 D5: 히트맵 클러스터링 분석
**미션**: 상관 패턴에 기반한 클러스터링으로 유전자/프로브 그룹을 발견해보세요!

**도전 과제**:
1. 유전자와 프로브를 각각 클러스터링하여 덴드로그램과 함께 히트맵 표시하기
2. 클러스터별로 다른 색상 사이드바 추가하기
3. 각 클러스터의 평균 상관패턴 비교 분석하기


In [ ]:
# 미션 D5 코드를 여기에 작성하세요!


## O2PLS 분석 (Two-Way Orthogonal Partial Least Squares)
- 공유 성분과 각 오믹스 고유 성분을 분리하여 분석합니다.
- PLSCanonical로 공유 변동을 추출하고, 잔차에서 직교 성분을 찾습니다.

In [ ]:
# 데이터 준비: 샘플×특성 형태로 전치
X = A.T.fillna(0.0)                         # (n_samples × n_genes)
Y = B.T.fillna(B.T.mean())                  # (n_samples × n_probes)

print(f"   📊 입력 데이터: X{X.shape}, Y{Y.shape}")

# 차원수 안전 제한 (샘플 수보다 작게)
n = X.shape[0]
Gcap = min(X.shape[1], max(5, n - 2), 200)  # 최소 5개, 최대 200개
Pcap = min(Y.shape[1], max(5, n - 2), 400)  # 최소 5개, 최대 400개

X = X.iloc[:, :Gcap] 
Y = Y.iloc[:, :Pcap]

print(f"   ⚙️  제한된 차원: X{X.shape}, Y{Y.shape}")

# 표준화
Xs = StandardScaler().fit_transform(X)
Ys = StandardScaler().fit_transform(Y)

In [ ]:
print("\nO2PLS-like 분석 수행 중...")

# Step 1: 공유 성분 추출 (PLSCanonical)
n_comp_shared = 3
print(f"공유 성분 개수: {n_comp_shared}개")

pls = PLSCanonical(n_components=n_comp_shared)
Xp, Yp = pls.fit_transform(Xs, Ys)                # 공유 점수  (samples × n_comp_shared)
X_load, Y_load = pls.x_loadings_, pls.y_loadings_ # 로딩 행렬

print("PLS 공유 성분 추출 완료")

# Step 2: 잔차 계산 (비공유 부분)
X_shared = Xp @ X_load.T  # 공유 부분 재구성
Y_shared = Yp @ Y_load.T
X_res = Xs - X_shared     # 잔차 (X 고유 부분)
Y_res = Ys - Y_shared     # 잔차 (Y 고유 부분)

print("잔차 계산 완료")

# Step 3: 직교 성분 추출 (각 오믹스의 고유 변동)
n_ortho_x, n_ortho_y = 2, 2
px = PCA(n_components=n_ortho_x).fit(X_res)
py = PCA(n_components=n_ortho_y).fit(Y_res)
Xo = px.transform(X_res)   # X-고유 직교 점수
Yo = py.transform(Y_res)   # Y-고유 직교 점수

print(f"직교 성분: mRNA {n_ortho_x}개, Methylation {n_ortho_y}개")

# Step 4: 공유 성분 품질 평가
shared_corr = np.array([np.corrcoef(Xp[:, i], Yp[:, i])[0, 1] for i in range(n_comp_shared)])

print("공유 성분 상관계수:")
for i, corr in enumerate(shared_corr, 1):
    print(f"Shared Component {i}: ρ = {corr:.3f}")

In [ ]:
print("📊 O2PLS-like 분석 시각화 시작...")

# 1행 3열 subplot
fig, axes = plt.subplots(1, 3, figsize=(24, 6))
fig.suptitle("O2PLS-like Shared Component Analysis", fontsize=18, fontweight='bold')

# ------------------------------
# Step 4: 공유 성분 상관계수 시각화
# ------------------------------
xs = np.arange(1, n_comp_shared + 1)
axes[0].plot(xs, shared_corr, marker="o", linewidth=2, markersize=8, color='darkgreen')
for i, v in enumerate(shared_corr, 1):
    axes[0].plot([i, i], [0, v], linewidth=3, alpha=0.6, color='darkgreen')
    axes[0].text(i, v + 0.03, f"{v:.3f}", ha="center", va="bottom", fontweight='bold')

axes[0].set_ylim(0, 1.05)
axes[0].set_xticks(xs)
axes[0].set_title("Shared Component Correlations", fontsize=14, fontweight='bold')
axes[0].set_xlabel("Shared Component")
axes[0].set_ylabel("Correlation Coefficient")
axes[0].grid(True, alpha=0.3)

# ------------------------------
# Step 5: 첫 번째 공유 성분 관계 (Hexbin)
# ------------------------------
hb = axes[1].hexbin(Xp[:, 0], Yp[:, 0], gridsize=30, alpha=0.8, cmap='viridis')
fig.colorbar(hb, ax=axes[1], label="Sample Density")

axes[1].set_title(
    f"Shared Component 1 Density\n(Correlation = {shared_corr[0]:.3f})",
    fontsize=14, fontweight='bold'
)
axes[1].set_xlabel("mRNA Shared Component 1")
axes[1].set_ylabel("Methylation Shared Component 1")
axes[1].grid(True, alpha=0.3)

# ------------------------------
# Step 6: 공유 성분 교차상관 히트맵
# ------------------------------
C_shared = np.corrcoef(Xp.T, Yp.T)[:n_comp_shared, n_comp_shared:]
sns.heatmap(
    pd.DataFrame(
        C_shared,
        index=[f"mRNA_S{i+1}" for i in range(n_comp_shared)],
        columns=[f"Meth_S{i+1}" for i in range(n_comp_shared)]
    ),
    annot=True,
    fmt=".3f",
    vmin=-1,
    vmax=1,
    cmap="RdBu_r",
    center=0,
    cbar_kws={'label': 'Cross-Correlation'},
    ax=axes[2]
)
axes[2].set_title("Shared Components Cross-Correlation Matrix",
                  fontsize=14, fontweight='bold')

# ------------------------------
# 레이아웃 정리 및 출력
# ------------------------------
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()

print("✅ O2PLS-like 분석 완료!")


### Vibe Coding 미션 D6: O2PLS 파라미터 최적화
**미션**: 서로 다른 파라미터 설정으로 O2PLS 분석을 수행하고 결과를 비교해보세요!

**도전 과제**:
1. 공유 성분 개수를 다르게 설정하여 분석 비교하기 (n_comp_shared = 2, 3, 5)
2. 직교 성분 개수를 조정하여 분석 결과 차이 확인하기 (n_ortho_x, n_ortho_y 변경)
3. 피처 개수를 달리하여 분석 성능 비교하기 (상위 50개 vs 200개 유전자)
4. 각 설정에서 공유 성분 상관계수와 설명력 비교 분석하기


In [ ]:
# 미션 D6 코드를 여기에 작성하세요!

